In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
from astropy.wcs import WCS
import numpy as np
from matplotlib.ticker import MultipleLocator
from scipy.stats import linregress
from matplotlib.colors import LogNorm
import astropy.io.fits as fits
from reproject import reproject_from_healpix
from matplotlib import cm
from matplotlib.colors import Normalize 
from matplotlib import pyplot as plt, patches
from scipy.interpolate import interpn
from reproject import reproject_to_healpix
from astropy_healpix import HEALPix
from astropy.coordinates import SkyCoord, ICRS, Galactic
from astropy import units as units
import matplotlib.cm as cm
import sys
from importlib import reload
import astropy.units as u
import copy
from matplotlib.ticker import MultipleLocator, FuncFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import gc
import copy

In [ ]:
#directory = '/srv/data/dva/dva_map_test_results/Anna/Apr8/'
directory = '/home/aordog/DATA/DVA/'

dir_out = '/home/aordog/TomFest_GMIMS/PLOTS/'
plot_name = 'DRAO_15m_540_1000_MHz_June23_2023'
plot_title = 'DRAO 15-m 538 - 1016 MHz'
basefilename = 'DVA_June18'

# Some functions:

In [ ]:
def zero_moment(hdr,data,peakthresh,maxFD,RM_arr):
    
    dFD = hdr['CDELT2']
    print('Calculating zeroth moment...')    
    print('Max FD to include: ',maxFD)
    print('FD channel width: ',dFD)
    
    zero_mom_hdr = hdr.copy()    
    zero_mom_hdr['NAXIS'] = 1  
    del zero_mom_hdr['CRVAL2']
    del zero_mom_hdr['CTYPE2']
    del zero_mom_hdr['CRPIX2']
    del zero_mom_hdr['CDELT2']
    del zero_mom_hdr['CUNIT2']
    del zero_mom_hdr['NAXIS2']
    try:
        del zero_mom_hdr['LAMSQ0']
    except:
        pass
    
    numpix = data.shape[1]   
    zero_mom_data = np.empty(numpix)
       
    for i in range(0,numpix):
        if i==int(numpix/2):
            print('halfway')
        #wgood = np.where((data[:,i]>=peakthresh) & (data[:,i]>=0.15*peakPI_data[i]) & (abs(RM_arr)<=maxFD))
        wgood = np.where((data[:,i]>=peakthresh) & (abs(RM_arr)<=maxFD))
        if np.size(wgood) != 0:
            zero_mom_data[i] = dFD*np.sum(data[wgood,i])
        else:
            zero_mom_data[i] = np.nan      
    
    print('done')
    print('')
    return(zero_mom_data,zero_mom_hdr)

def first_moment(hdr,data,peakthresh,maxFD,RM_arr,zero_mom):
    
    dFD = hdr['CDELT2']
    print('Calculating first moment...')    
    print('Max FD to include: ',maxFD)
    print('FD channel width: ',dFD)
    
    first_mom_hdr = hdr.copy()    
    first_mom_hdr['NAXIS'] = 1  
    del first_mom_hdr['CRVAL2']
    del first_mom_hdr['CTYPE2']
    del first_mom_hdr['CRPIX2']
    del first_mom_hdr['CDELT2']
    del first_mom_hdr['CUNIT2']
    del first_mom_hdr['NAXIS2']
    try:
        del zero_mom_hdr['LAMSQ0']
    except:
        pass
    
    numpix = data.shape[1]   
    first_mom_data = np.empty(numpix)
       
    for i in range(0,numpix):
        if i==int(numpix/2):
            print('halfway')
        #wgood = np.where((data[:,i]>=peakthresh) & (data[:,i]>=0.15*peakPI_data[i]) & (abs(RM_arr)<=maxFD))
        wgood = np.where((data[:,i]>=peakthresh) & (abs(RM_arr)<=maxFD))
        if np.size(wgood) != 0:
            first_mom_data[i] = dFD*np.sum(data[wgood,i]*RM_arr[wgood])/zero_mom[i]
        else:
            first_mom_data[i] = np.nan      
    
    print('done')
    print('')
    return(first_mom_data,first_mom_hdr)


def second_moment(hdr,data,peakthresh,maxFD,RM_arr,zero_mom,first_mom):
    
    dFD = hdr['CDELT2']
    print('Calculating second moment...')    
    print('Max FD to include: ',maxFD)
    print('FD channel width: ',dFD)
    
    second_mom_hdr = hdr.copy()    
    second_mom_hdr['NAXIS'] = 1  
    del second_mom_hdr['CRVAL2']
    del second_mom_hdr['CTYPE2']
    del second_mom_hdr['CRPIX2']
    del second_mom_hdr['CDELT2']
    del second_mom_hdr['CUNIT2']
    del second_mom_hdr['NAXIS2']
    #del second_mom_hdr['LAMSQ0']
    
    numpix = data.shape[1]   
    second_mom_data = np.empty(numpix)
       
    for i in range(0,numpix):
        if i==int(numpix/2):
            print('halfway')
        #wgood = np.where((data[:,i]>=peakthresh) & (data[:,i]>=0.15*peakPI_data[i]) & (abs(RM_arr)<=maxFD))
        wgood = np.where((data[:,i]>=peakthresh) & (abs(RM_arr)<=maxFD))
        if np.size(wgood) != 0:
            second_mom_data[i] = dFD*np.sum( data[wgood,i] * (RM_arr[wgood] - first_mom[i])**2 )/zero_mom[i]
        else:
            second_mom_data[i] = np.nan      
    
    print('done')
    print('')
    return(second_mom_data,second_mom_hdr)


class nice_map_zoom(object):
       
    def mollweide(data,coordsys,Pmin,Pmax,pixsize=0.5,fs=14,cmap=cm.viridis,title='',bg_clr='gray',
               xc=180,yc=0,grid_clr='black',lbl_clr='black',axtitles=True,cbarlbl='',lonra=[-180,180],latra=[-90,90],
               *args, **kwargs):
        print('Mollweide')
        proj = '-MOL'
        fc = 'EllipticalFrame'
        hdr_new,cs,ny,nx = make_new_header_zoom(coordsys,proj,pixsize,xc,yc,lonra)
        make_the_map_zoom(data,hdr_new,fc,cs,Pmin,Pmax,fs,cmap,title,bg_clr,grid_clr,lbl_clr,axtitles,cbarlbl,proj,latra,ny,nx)
        return
    
    def platcar(data,coordsys,Pmin,Pmax,pixsize=0.5,fs=14,cmap=cm.viridis,title='',bg_clr='gray',
               xc=180,yc=0,grid_clr='black',lbl_clr='black',axtitles=True,cbarlbl='',lonra=[-180,180],latra=[-90,90],
               *args, **kwargs):
        print('Plate Caree')
        proj = '-CAR'
        fc = 'Rect'
        hdr_new,cs,ny,nx = make_new_header_zoom(coordsys,proj,pixsize,xc,yc,lonra)
        make_the_map_zoom(data,hdr_new,fc,cs,Pmin,Pmax,fs,cmap,title,bg_clr,grid_clr,lbl_clr,axtitles,cbarlbl,proj,latra,ny,nx)
        return

def make_new_header_zoom(coordsys,proj,pixsize,xc,yc,lonra):
    if coordsys == 'C':
        ctype1 = 'RA--'+proj
        ctype2 = 'DEC-'+proj
        cs = 'icrs'
    if coordsys == 'G':
        ctype1 = 'GLON'+proj
        ctype2 = 'GLAT'+proj
        cs = 'galactic'

    if (proj == '-MOL') or (proj == '-AIT'):
        nx = int(round((((lonra[1]-lonra[0])/360)*4*np.sqrt(2)*180/(pixsize*np.pi)),0))
        ny = int(round((2*np.sqrt(2)*180/(pixsize*np.pi)),0))
    if (proj == '-CAR'):
        nx = int(round(((lonra[1]-lonra[0])/360)*(360/pixsize),0))
        ny = int(round((180/pixsize),0))
    if (proj == '-TAN'):
        nx = int(round((15*180/(pixsize*np.pi)),0))
        ny = int(round((15*180/(pixsize*np.pi)),0))

    hdr_new = fits.Header.fromstring("""
NAXIS   =                    2
CUNIT1  = 'deg     '
CUNIT2  = 'deg     '
COORDSYS= 'icrs    '
""", sep='\n')
    hdr_new['NAXIS1']  = nx 
    hdr_new['NAXIS2']  = ny
    hdr_new['CTYPE1']  = ctype1
    hdr_new['CRPIX1']  = nx/2.+0.5 
    hdr_new['CRVAL1']  = xc          
    hdr_new['CDELT1']  = -pixsize 
    hdr_new['CTYPE2']  = ctype2
    hdr_new['CRPIX2']  = ny/2.+0.5
    hdr_new['CRVAL2']  = yc
    hdr_new['CDELT2']  = pixsize
    #print(repr(hdr_new))    
    return(hdr_new,cs,ny,nx)    
    
    
def make_the_map_zoom(data,hdr_new,fc,cs,Pmin,Pmax,fs,cmap,title,bg_clr,
                 grid_clr,lbl_clr,axtitles,cbarlbl,proj,latra,ny,nx):

    array, footprint = reproject_from_healpix((data,cs),hdr_new, nested=False)
    
    fig = plt.figure(figsize=(15,10))
    
    srise = np.repeat(211.1,1000)
    sset = np.repeat(332.1,1000)
    dec_arr1 = np.linspace(14.7,86,1000)
    dec_arr2 = np.linspace(12,86,1000)

    c1 = SkyCoord(ra=srise*u.degree, dec=dec_arr1*u.degree, frame='icrs')
    c2 = SkyCoord(ra=sset*u.degree, dec=dec_arr2*u.degree, frame='icrs')

    c1_lon = c1.galactic.l.deg
    c1_lon[np.where(c1_lon >=180)] = c1_lon[np.where(c1_lon >=180)]-360.
    c1_lat = c1.galactic.b.deg
    c2_lon = c2.galactic.l.deg
    c2_lon[np.where(c2_lon >=180)] = c2_lon[np.where(c2_lon >=180)]-360.
    c2_lat = c2.galactic.b.deg
    
    
    if fc == 'EllipticalFrame':
        ax = plt.subplot(111, projection=WCS(hdr_new))
    else:
        ax = plt.subplot(111, projection=WCS(hdr_new))
                
    cmap.set_bad(color=bg_clr)
    im = ax.imshow(array, vmin=Pmin, vmax=Pmax,cmap=cmap)
    #ax.plot((360-c1_lon-120)*nx/240,(c1_lat+90.)*ny/180,color='black',linestyle='dashed')
    #ax.plot((360-c2_lon-120)*nx/240,(c2_lat+90.)*ny/180,color='black',linestyle='dashed')
    
    ax.coords.grid(color=grid_clr)
    ax.coords[0].set_ticklabel(color=lbl_clr,fontsize=fs)
    ax.coords[1].set_ticklabel(color=lbl_clr,fontsize=fs)
    ax.coords[0].set_ticks(number=5)
    ax.coords[1].set_ticks(number=5)
    ax.set_ylim((latra[0]+90)*ny/180,(latra[1]+90)*ny/180)
    #x_lim = ax.get_xlim()
    #y_lim = ax.get_ylim()
    #print(y_lim)
    #print(y_lim[0]*180/ny,y_lim[1]*180/ny)
    cbar = plt.colorbar(im,orientation='horizontal',shrink=0.4,pad=0.1)
    cbar.ax.tick_params(labelsize=fs)
    cbar.set_label(cbarlbl,fontsize=fs)
    ax.set_title(title,fontsize=fs+2)
    if axtitles:
        if (proj == '-MOL') or (proj == '-AIT'):
            ypad = 1
        else:
            ypad = 1
        if cs == 'icrs':            
            ax.coords[0].set_axislabel('Right Ascension',fontsize=fs,minpad=ypad)
            #ax.coords[0].set_axislabel_position('t')
            ax.coords[1].set_axislabel('Declination',fontsize=fs)
        if cs == 'galactic':
            ax.coords[0].set_axislabel('Galactic Longitude',fontsize=fs,minpad=ypad)
            ax.coords[1].set_axislabel('Galactic Latitude',fontsize=fs)
    
    return


# 1 Run RM synthesis on FITS cubes

In [ ]:
run_rm_synth = False

#if run_rm_synth:
#    !/home/ordoga/Python/RM-Tools/RMtools_3D/do_RMsynth_3D.py \
#     /home/ordoga/DATA/DVA/New_mini_survey_q_cube.fits \
#     /home/ordoga/DATA/DVA/New_mini_survey_u_cube.fits \
#     /home/ordoga/DATA/DVA/ListOfFrequencies_Hz.txt -v -d 1 -l 200
    
if run_rm_synth:
    !/home/aordog/Python/RM-Tools-master/RMtools_3D/do_RMsynth_3D.py \
     /home/aordog/DATA/DVA/New_mini_survey_q_cube.fits \
     /home/aordog/DATA/DVA/New_mini_survey_u_cube.fits \
     /home/aordog/DATA/DVA/ListOfFrequencies_Hz.txt -v -d 1 -l 200

In [ ]:
!/home/aordog/Python/RM-Tools-master/RMtools_3D/do_RMclean_3D.py \
/home/aordog/DATA/DVA/FDF_tot_dirty.fits \
/home/aordog/DATA/DVA/RMSF_tot.fits -c 0.3 -v



# 2 Convert RM synth files to Healpix:

In [ ]:
convert_to_hpx = True # Make this False if already have Healpix files


## 2.1 Read in RM synthesis files:

In [ ]:
if convert_to_hpx:

    hdu_FDcube = fits.open(directory+'FDF_tot_dirty.fits')
    hdr_FDcube = hdu_FDcube[0].header
    data_FDcube = hdu_FDcube[0].data
    print(data_FDcube.shape)

    hdu_PIpeak = fits.open(directory+'FDF_maxPI.fits')
    hdr_PIpeak = hdu_PIpeak[0].header
    data_PIpeak = hdu_PIpeak[0].data[0]
    print(data_PIpeak.shape)
    
    #hdu_FDcube_CL = fits.open(directory+'FDF_clean_tot.fits')
    #hdr_FDcube_CL = hdu_FDcube_CL[0].header
    #data_FDcube_CL = hdu_FDcube_CL[0].data
    #print(data_FDcube_CL.shape)

## 2.2 Make new 2D header out of PI peak file

In [ ]:
if convert_to_hpx:

    hdr_2D_new = hdr_PIpeak.copy()
    hdr_2D_new['NAXIS'] = 2
    del hdr_2D_new['NAXIS3']
    del hdr_2D_new['CTYPE3']
    del hdr_2D_new['CDELT3']
    del hdr_2D_new['CRVAL3']
    del hdr_2D_new['CUNIT3']
    del hdr_2D_new['CRPIX3']
    print(repr(hdr_2D_new))

## 2.3 Reproject to healpix

In [ ]:
if convert_to_hpx:

    nside = 128 # Note: smoothing a bit - original was nside=64 for now

    FDcube_hpx = np.empty([data_FDcube_CL.shape[0],12*nside**2])

    for i in range(0,data_FDcube_CL.shape[0]):
        print(i)    
        FDcube_hpx[i,:], footprint = reproject_to_healpix((data_FDcube_CL[i,:,:], hdr_2D_new), 
                                                          'icrs', nside=nside)
    
    #PIpeak_hpx, footprint = reproject_to_healpix((data_PIpeak, hdr_2D_new), 'icrs', nside=nside)
    

## 2.4 Write out new healpix files

In [ ]:
convert_to_hpx = True
nside = 128
if convert_to_hpx:
    
    hdu_new_hpx = fits.PrimaryHDU()
    hdr_hpx = hdu_new_hpx.header
    
    hdr_hpx['PIXTYPE'] = ('HEALPIX ', 'HEALPIX pixelisation')
    hdr_hpx['ORDERING'] = ('RING', 'Pixel ordering scheme, either RING or NESTED')
    hdr_hpx['NSIDE'] = (nside,     'Resolution parameter of HEALPIX')
    hdr_hpx['INDXSCHM'] = ('IMPLICIT', 'Indexing: IMPLICIT or EXPLICIT')
    hdr_hpx['OBJECT'] = ('FULLSKY ', 'Sky coverage, either FULLSKY or PARTIAL')
    hdr_hpx['COORDSYS'] = ('C', 'Ecliptic, Galactic or Celestial (equatorial)')
    hdr_hpx['NAXIS'] = 2
    hdr_hpx['NAXIS2'] = hdr_FDcube['NAXIS3']   
    hdr_hpx['CRPIX2'] = hdr_FDcube['CRPIX3']
    hdr_hpx['CDELT2'] = hdr_FDcube['CDELT3']
    hdr_hpx['CUNIT2'] = 'MHz'
    hdr_hpx['CTYPE2'] = 'FREQ'
    hdr_hpx['CRVAL2'] = hdr_FDcube['CRVAL3']

    hdr_hpx_1D = hdr_hpx.copy()
    hdr_hpx_1D['NAXIS'] = 1
    del hdr_hpx_1D['NAXIS2']
    del hdr_hpx_1D['CRPIX2']
    del hdr_hpx_1D['CDELT2']
    del hdr_hpx_1D['CTYPE2']
    del hdr_hpx_1D['CRVAL2']
    del hdr_hpx_1D['CUNIT2']
    
    print('')
    print('HEADER FOR FD CUBE')
    print('--------------')
    print(repr(hdr_hpx))
    print('')
    print('HEADER FOR PI PEAKS')
    print('---------------')
    print(repr(hdr_hpx_1D))
    
    #fits.writeto(directory+basefilename+'_FDcube.hpx.fits',FDcube_hpx,
    #             header=hdr_hpx,overwrite=True,output_verify='fix')
    
    fits.writeto(directory+basefilename+'_FDcube_CLEAN.hpx.fits',FDcube_hpx,
                 header=hdr_hpx,overwrite=True,output_verify='fix')
    
    #fits.writeto(directory+basefilename+'_peakPI.hpx.fits',PIpeak_hpx,
    #             header=hdr_hpx_1D,overwrite=True,output_verify='fix')

# 3 Work with Healpix FD files

## 3.1 Read in healpix files

In [ ]:
hdu_FDcube_hpx = fits.open(directory+basefilename+'_FDcube.hpx.fits')
hdr_FDcube_hpx = hdu_FDcube_hpx[0].header
data_FDcube_hpx = hdu_FDcube_hpx[0].data
print(data_FDcube_hpx.shape)

#hdu_PIpeak_hpx = fits.open(directory+basefilename+'_peakPI.hpx.fits')
#hdr_PIpeak_hpx = hdu_PIpeak_hpx[0].header
#data_PIpeak_hpx = hdu_PIpeak_hpx[0].data
#print(data_PIpeak_hpx.shape)

FD_idx = np.linspace(0,hdr_FDcube_hpx['NAXIS2']-1,hdr_FDcube_hpx['NAXIS2'])
FD_ax = hdr_FDcube_hpx['CRVAL2']+(FD_idx - hdr_FDcube_hpx['CRPIX2']+1)*hdr_FDcube_hpx['CDELT2']

print(FD_ax.shape)

## 3.2 Calculate Moments

In [ ]:
maxabsFD = 60
minPI = 0

M0,M0_hdr = zero_moment(hdr_FDcube_hpx,data_FDcube_hpx,minPI,maxabsFD,FD_ax)
M1,M1_hdr = first_moment(hdr_FDcube_hpx,data_FDcube_hpx,minPI,maxabsFD,FD_ax,M0)
M2,M2_hdr = second_moment(hdr_FDcube_hpx,data_FDcube_hpx,minPI,maxabsFD,FD_ax,M0,M1)

## 3.3 Convert moment maps to Galactic

In [ ]:
import healpy
r = healpy.rotator.Rotator(coord=['C','G'])
M0_gal = r.rotate_map_pixel(M0)
M1_gal = r.rotate_map_pixel(M1)
M2_gal = r.rotate_map_pixel(M2)

In [ ]:
fits.writeto(directory+basefilename+'_M1.hpx.fits',M1_gal,
                 header=hdr_hpx,overwrite=True,output_verify='fix')

## 3.4 Make M1 map

In [ ]:
dlon = 120
loncent = 120
#mpl.cm.get_cmap("bwr").copy()

nice_map_zoom.platcar(M1_gal,'G',-10,10,pixsize=0.25,title=plot_title,
                 cmap = copy.copy(mpl.cm.get_cmap("bwr")),bg_clr='gray',
                 xc=loncent,yc=0,grid_clr='black',axtitles=True,cbarlbl='rad/m$^2$',
                 lonra=[loncent-dlon,loncent+dlon],latra=[-60,90])
plt.tight_layout()
plt.savefig(dir_out+plot_name+'_M1.jpg')

## 3.5 Make M0 map

In [ ]:
dlon = 120
loncent = 120

nice_map_zoom.platcar(M0_gal,'G',0,600,pixsize=0.25,title=plot_title,
                 cmap = copy.copy(mpl.cm.get_cmap("viridis")),bg_clr='gray',
                 xc=loncent,yc=0,grid_clr='black',axtitles=True,cbarlbl='Jy/beam/RMSF rad/m$^2$',
                 lonra=[loncent-dlon,loncent+dlon],latra=[-60,90])
plt.tight_layout()
plt.savefig(dir_out+plot_name+'_M0_v2.jpg')


In [ ]:
dlon = 120
loncent = 120

nice_map_zoom.platcar(np.sqrt(M2_gal),'G',20,40,pixsize=0.25,title=plot_title,
                 cmap = copy.copy(mpl.cm.get_cmap("viridis")),bg_clr='gray',
                 xc=loncent,yc=0,grid_clr='black',axtitles=True,cbarlbl='rad/m$^2$',
                 lonra=[loncent-dlon,loncent+dlon],latra=[-60,90])
plt.tight_layout()
plt.savefig(dir_out+plot_name+'_M2.jpg')

In [ ]:
f1 = 538e6
f2 = 1016e6

dlbd2 = ((3e8)/f1)**2 - ((3e8)/f2)**2

print(3.79/dlbd2)

# 4 Faraday spectrum analysis

## 4.1 Read in CHIME and GMIMS

In [ ]:
# CHIME (Equatorial):
hdu_FDcube_hpx_CH = fits.open('/home/aordog/DATA/CHIME/FDF_tot_dirty_Oct132022.hpx.fits')
#hdu_FDcube_hpx_CH = fits.open('/srv/aordog/CHIME_data/CHIME_Fall_2022/RM_SYNTH_v3/FDF_tot_dirty_Oct132022.hpx.fits')
hdr_FDcube_hpx_CH = hdu_FDcube_hpx_CH[0].header
data_FDcube_hpx_CH = hdu_FDcube_hpx_CH[0].data
print(data_FDcube_hpx_CH.shape)

FD_idx_CH = np.linspace(0,hdr_FDcube_hpx_CH['NAXIS2']-1,hdr_FDcube_hpx_CH['NAXIS2'])
FD_ax_CH = hdr_FDcube_hpx_CH['CRVAL2']+(FD_idx_CH - hdr_FDcube_hpx_CH['CRPIX2']+1)*hdr_FDcube_hpx_CH['CDELT2']

# GMIMS (Galactic):
hdu_FDcube_hpx_G = fits.open('/home/aordog/DATA/GMIMS_HBN_download/GMIMS-HBN_v1_gal_hpx_FD_PI.fits')
#hdu_FDcube_hpx_G = fits.open('/srv/data/gmims/gmims-hbn/GMIMS-HBN_v1_gal_hpx_FD_PI.fits')
hdr_FDcube_hpx_G = hdu_FDcube_hpx_G[0].header
data_FDcube_hpx_G = hdu_FDcube_hpx_G[0].data
print(data_FDcube_hpx_G.shape)

FD_idx_G = np.linspace(0,hdr_FDcube_hpx_G['NAXIS2']-1,hdr_FDcube_hpx_G['NAXIS2'])
FD_ax_G = hdr_FDcube_hpx_G['CRVAL2']+(FD_idx_G - hdr_FDcube_hpx_G['CRPIX2']+1)*hdr_FDcube_hpx_G['CDELT2']

# QUAIL (Equatorial)
hdu_FDcube_hpx = fits.open(directory+basefilename+'_FDcube_CLEAN.hpx.fits')
hdr_FDcube_hpx = hdu_FDcube_hpx[0].header
data_FDcube_hpx = hdu_FDcube_hpx[0].data
print(data_FDcube_hpx.shape)

FD_idx = np.linspace(0,hdr_FDcube_hpx['NAXIS2']-1,hdr_FDcube_hpx['NAXIS2'])
FD_ax = hdr_FDcube_hpx['CRVAL2']+(FD_idx - hdr_FDcube_hpx['CRPIX2']+1)*hdr_FDcube_hpx['CDELT2']


In [ ]:
print(repr(hdr_FDcube_hpx))

## 4.2 Rotate DVA and CHIME to Galactic

In [ ]:
import healpy
r = healpy.rotator.Rotator(coord=['C','G'])
data_FDcube_hpx_CH_gal = np.array(r.rotate_map_pixel(data_FDcube_hpx_CH))
print(data_FDcube_hpx_CH_gal.shape)

In [ ]:
data_FDcube_hpx_gal = np.array(r.rotate_map_pixel(data_FDcube_hpx))
print(data_FDcube_hpx_gal.shape)

In [ ]:
dlon = 120
loncent = 120

nice_map_zoom.platcar(data_FDcube_hpx_gal[200,:],'G',0,5,pixsize=0.25,title=plot_title,
                 cmap = copy.copy(mpl.cm.get_cmap("viridis")),bg_clr='gray',
                 xc=loncent,yc=0,grid_clr='black',axtitles=True,cbarlbl='Jy/beam/RMSF rad/m$^2$',
                 lonra=[loncent-dlon,loncent+dlon],latra=[-60,90])

In [ ]:
import astropy_healpix

In [ ]:
hpx_gal_256 = astropy_healpix.HEALPix(nside=256, order='ring', frame='galactic')
hpx_gal_128 = astropy_healpix.HEALPix(nside=128, order='ring', frame='galactic')

In [ ]:
hdu_H = fits.open('/home/aordog/Travel/Sweden_Vancouver_2023/data/sky_maps.fits')
hdr_H = hdu_H[1].header
FD_H = hdu_H[1].data

#print(repr(hdr_H))
#print(FD_H.shape)

In [ ]:
ltest = 50.0
btest = 65.

fig, ax = plt.subplots(1,1,figsize=(12,6))
fs = 14

idx_CH_G = hpx_gal_256.skycoord_to_healpix(SkyCoord(frame="galactic",l=ltest*u.deg,b=btest*u.deg))
idx_DVA = hpx_gal_128.skycoord_to_healpix(SkyCoord(frame="galactic",l=ltest*u.deg,b=btest*u.deg))

ax.plot(FD_ax,data_FDcube_hpx_gal[:,idx_DVA]/np.nanmax(data_FDcube_hpx_gal[:,idx_DVA]))
ax.plot(FD_ax_CH,data_FDcube_hpx_CH_gal[:,idx_CH_G]/np.nanmax(data_FDcube_hpx_CH_gal[:,idx_CH_G]))
ax.plot(FD_ax_G,data_FDcube_hpx_G[:,idx_CH_G]/np.nanmax(data_FDcube_hpx_G[:,idx_CH_G]))
ax.set_xlim(-200,200)
ax.set_ylim(0,1.1)
#ax.set_ylim(bottom=0)
#ax.set_xlim(-150,150)
ax.tick_params(axis="x", labelsize=fs)
ax.tick_params(axis="y", labelsize=fs)
ax.grid()

45,  40
75,  60
150, 30
178, 12
150, 24
150, 23.5

In [ ]:
lplot = [45.,75.,150.,189.,150.,150.]
bplot = [40.,60.,30.,50.,24.,23.5]

fig, axs = plt.subplots(3,2,figsize=(14,10))
fs = 14

for i in range(0,3):
    for j in range(0,2):
        ii = i*2+j
        
        idx_CH_G = hpx_gal_256.skycoord_to_healpix(SkyCoord(frame="galactic",l=lplot[ii]*u.deg,b=bplot[ii]*u.deg))
        idx_DVA = hpx_gal_128.skycoord_to_healpix(SkyCoord(frame="galactic",l=lplot[ii]*u.deg,b=bplot[ii]*u.deg))

        axs[i,j].plot(FD_ax,data_FDcube_hpx_gal[:,idx_DVA]/np.nanmax(data_FDcube_hpx_gal[:,idx_DVA]),label='DRAO 15-m')
        axs[i,j].plot(FD_ax_CH,data_FDcube_hpx_CH_gal[:,idx_CH_G]/np.nanmax(data_FDcube_hpx_CH_gal[:,idx_CH_G]),label='CHIME')
        axs[i,j].plot(FD_ax_G,data_FDcube_hpx_G[:,idx_CH_G]/np.nanmax(data_FDcube_hpx_G[:,idx_CH_G]),label='GMIMS-HBN')

        axs[i,j].set_title(r'$\ell$='+str(lplot[ii])+'$^{\circ}$, $b$='+str(bplot[ii])+'$^{\circ}$',
                           fontsize=fs+2)
        axs[i,j].grid()
        axs[i,j].set_ylim(bottom=0)
        axs[i,j].set_xlim(-150,150)
        axs[i,j].tick_params(axis="x", labelsize=fs)
        axs[i,j].tick_params(axis="y", labelsize=fs)

axs[0,0].legend(fontsize=fs)
axs[2,1].set_xlabel(r'Faraday Depth (rad m$^{-2}$)',fontsize=fs)
axs[2,0].set_xlabel(r'Faraday Depth (rad m$^{-2}$)',fontsize=fs)
axs[0,0].set_ylabel(r'PI / PI$_{peak}$',fontsize=fs)
axs[1,0].set_ylabel(r'PI / PI$_{peak}$',fontsize=fs)
axs[2,0].set_ylabel(r'PI / PI$_{peak}$',fontsize=fs)

#fig.suptitle(title,fontsize=fs)
plt.tight_layout()
#plt.savefig('/home/aordog/DVA_PLOTS/CHIME_GMIMS_DVA_spectra_examples.png')
plt.savefig('/home/aordog/TomFest_GMIMS/CHIME_GMIMS_DVA_spectra_examples_new.png')


In [ ]:
def make_new_header(hdr,pixsize,xc,yc):
    
    #coordsys = hdr['COORDSYS']
    
    #if coordsys == 'C':
    #    ctype1 = 'RA---CAR'
    #    ctype2 = 'DEC--CAR'
    #    cs = 'icrs'
    #if coordsys == 'G':
    #    ctype1 = 'GLON-CAR'
    #    ctype2 = 'GLAT-CAR'
    #    cs = 'galactic'
        
    ctype1 = 'GLON-CAR'
    ctype2 = 'GLAT-CAR'
    cs = 'galactic'

    nx = int(round((360/pixsize),0))
    ny = int(round((180/pixsize),0))

    hdr_new = fits.Header.fromstring("""
NAXIS   =                    3
CUNIT1  = 'deg     '
CUNIT2  = 'deg     '
""", sep='\n')
    hdr_new['COORDSYS']  = cs
    hdr_new['NAXIS1']  = nx 
    hdr_new['NAXIS2']  = ny
    hdr_new['NAXIS3']  = hdr['NAXIS2']
    
    hdr_new['CTYPE1']  = ctype1
    hdr_new['CRPIX1']  = nx/2.+0.5 
    hdr_new['CRVAL1']  = xc          
    hdr_new['CDELT1']  = -pixsize
    
    hdr_new['CTYPE2']  = ctype2
    hdr_new['CRPIX2']  = ny/2.+0.5
    hdr_new['CRVAL2']  = yc
    hdr_new['CDELT2']  = pixsize
    
    hdr_new['CTYPE3']  = 'FDEP'#hdr['CTYPE2']
    hdr_new['CRPIX3']  = hdr['CRPIX2']
    hdr_new['CRVAL3']  = hdr['CRVAL2']#*1e6
    hdr_new['CDELT3']  = hdr['CDELT2']#*1e6
    hdr_new['CUNIT3']  = 'rad/m^2'
    
    hdr_new2D = hdr_new.copy()
    del hdr_new2D['NAXIS3']
    del hdr_new2D['CTYPE3']
    del hdr_new2D['CRPIX3']
    del hdr_new2D['CRVAL3']
    del hdr_new2D['CDELT3']
    del hdr_new2D['CUNIT3']
    hdr_new2D['NAXIS'] = 2
    
    return(hdr_new,hdr_new2D,cs)

## Write out FITS files:

In [ ]:
hdr_new, hdr_new2D, cs = make_new_header(hdr_FDcube_hpx,0.5,0,0)
print(repr(hdr_new))
print('')
print(repr(hdr_new2D))

data_new = np.empty([hdr_new['NAXIS3'],hdr_new['NAXIS2'],hdr_new['NAXIS1']])

for i in range(0,data_new.shape[0]):
    data_new[i,:,:], footprint = reproject_from_healpix((data_FDcube_hpx_gal[i],cs),
                                                        hdr_new2D, nested=False)
    print(i)
    
fits.writeto('/home/aordog/TomFest_GMIMS/FD_cube_DRAGONS_CLEAN.fits', 
             data_new.astype('float32'), hdr_new, overwrite=True)



In [ ]:
maxabsFD = 60
minPI = 0

M0_CH,M0_hdr_CH = zero_moment(hdr_FDcube_hpx_CH,data_FDcube_hpx_CH,minPI,
                              maxabsFD,FD_ax_CH)
M1_CH,M1_hdr_CH = first_moment(hdr_FDcube_hpx_CH,data_FDcube_hpx_CH,minPI,
                               maxabsFD,FD_ax_CH,M0_CH)
M2_CH,M2_hdr_CH = second_moment(hdr_FDcube_hpx_CH,data_FDcube_hpx_CH,minPI,
                                maxabsFD,FD_ax_CH,M0_CH,M1_CH)



In [ ]:
import healpy
r = healpy.rotator.Rotator(coord=['C','G'])
M0_gal_CH = r.rotate_map_pixel(M0_CH)
M1_gal_CH = r.rotate_map_pixel(M1_CH)
M2_gal_CH = r.rotate_map_pixel(M2_CH)

In [ ]:
dlon = 120
loncent = 120
#mpl.cm.get_cmap("bwr").copy()

nice_map_zoom.platcar(M1_gal_CH,'G',-10,10,pixsize=0.25,title='CHIME 600-800 MHz M1',
                 cmap = copy.copy(mpl.cm.get_cmap("bwr")),bg_clr='gray',
                 xc=loncent,yc=0,grid_clr='black',axtitles=True,cbarlbl='rad/m$^2$',
                 lonra=[loncent-dlon,loncent+dlon],latra=[-60,90])
plt.tight_layout()
plt.savefig(dir_out+plot_name+'_M1_CH.jpg')

In [ ]:
dlon = 120
loncent = 120

nice_map_zoom.platcar(M0_gal_CH,'G',0,30,pixsize=0.25,title='CHIME 600-800 MHz M0',
                 cmap = copy.copy(mpl.cm.get_cmap("viridis")),bg_clr='gray',
                 xc=loncent,yc=0,grid_clr='black',axtitles=True,cbarlbl='Jy/beam/RMSF rad/m$^2$',
                 lonra=[loncent-dlon,loncent+dlon],latra=[-60,90])
plt.tight_layout()
plt.savefig(dir_out+plot_name+'_M0_CH.jpg')


In [ ]:
dlon = 120
loncent = 120

nice_map_zoom.platcar(np.sqrt(M2_gal_CH),'G',15,40,pixsize=0.25,title='CHIME 600-800 MHz M2',
                 cmap = copy.copy(mpl.cm.get_cmap("viridis")),bg_clr='gray',
                 xc=loncent,yc=0,grid_clr='black',axtitles=True,cbarlbl='rad/m$^2$',
                 lonra=[loncent-dlon,loncent+dlon],latra=[-60,90])
plt.tight_layout()
plt.savefig(dir_out+plot_name+'_M2_CH.jpg')


In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u

def loops(c='k', text=True, text_c='k', **kwargs):
    # Names and coordinates from Vidal+ 2015
    loop_names = ['I', 'II', 'III', 'IV', 'V', 'VI', 'GCS', 'IIIs', 'VIIa', 'VIIb', 'IX', 'X',
                'XI', 'XII'
                ]
    l_loops = [
        329, 100.0, 124, 315.8, 127.5, 120.5, 344.0, 106, 345, 0.7, 332, 30, 227, 300
    ]
    b_loops = [
        17.5, -32.5, 15.5, 48.1, 18, 30.8, 4.8, -22, 0, -23.3, 16, 35, 38, 0.7
    ]
    r_loops = [
        58, 45.5, 32.5, 19.3, 67.2, 72.4, 18.5, 25, 65, 45.9, 46.5, 67, 81, 27.6
    ]
    loop_dict = {}
    for name, l, b, r in zip(loop_names, l_loops, b_loops, r_loops):
        theta = np.linspace(0, 2*np.pi, 1000)
        c = SkyCoord(l*u.deg, b*u.deg, frame='galactic')

        pa = np.rad2deg(theta) * u.deg
        sep = r* u.deg
        coord = c.directional_offset_by(pa, sep)  

        l_circ_l = coord.l.degree
        b_circ_l = coord.b.degree
        loop_dict.update(
            {
                name: {
                    'l': l,
                    'b': b,
                    'r': r,
                    'l_circ': l_circ_l,
                    'b_circ': b_circ_l
                }
            }
        )
    props = dict(boxstyle='round', facecolor='w', alpha=0.9)

    # Loop through Loops that we care about
    for name in ['I', 'II', 'III', 'IIIs']:
        # Bad code to add text along the loop
        if name == 'I':
            idx = 650
        elif name == 'II':
            idx = 450
        elif name == 'III':
#             idx = 200
            idx = 150
        elif name == 'IIIs':
            idx = 300
        else:
            idx = 450
        if name == 'I':
            horizontalalignment='right'
            verticalalignment='baseline'
        else:
            horizontalalignment='left'
            verticalalignment='center'
        if text:
            hp.projtext(loop_dict[name]['l_circ'][idx], loop_dict[name]['b_circ'][idx], 
                        f'Loop {name}', color=text_c, lonlat=True, coord='G', 
                        horizontalalignment=horizontalalignment,
                       verticalalignment=verticalalignment,
                       alpha = 1, bbox=props)
        hp.projplot(loop_dict[name]['l_circ'], loop_dict[name]['b_circ'], 
                   lonlat=True, coord='G', **kwargs)

In [ ]:
import healpy as hp

In [ ]:
hp.orthview(M1_gal,half_sky=True,rot=(120,50),cmap='RdBu_r',min=-10,max=10)
hp.graticule()

In [ ]:
hp.mollview(M1_gal,rot=(120,0),cmap='RdBu_r',min=-10,max=10)
hp.graticule()
loops(color='k')

In [ ]:
hp.mollview(M0_gal,rot=(120,0),cmap='viridis',min=0,max=400)
hp.graticule()
loops(color='k')